In [1]:
import numpy as np
import pandas as pd

In [2]:
from concurrent.futures import ProcessPoolExecutor

In [3]:
df = pd.read_csv('stocks.csv')[['ticker','date','ret']]

In [4]:
df.date = df.date//100

In [5]:
df = df[df.date>201000]

In [6]:
df['ret'] = df['ret']*100

In [7]:
n = 28

In [8]:
sample = []
for stock in set(df.ticker):
        x = df[df.ticker==stock].set_index('date').dropna()
        if len(x)>120:
            sample.append(x[['ret']].rename({'ret':stock},axis=1))

In [9]:
df = pd.concat(sample,axis=1)

In [10]:
def ret_var(w):
    std = np.sqrt(df.cov().mul(w,axis=0).mul(w,axis=1).sum().sum())
    eret = df.mul(w,axis=1).sum(axis=1).mean()*12
    return eret,std,w

In [18]:
def compute(_):
    r = np.random.RandomState()
    w = r.rand(n) - r.rand(n)
    w = w / w.sum()
    if w.max()<3 and w.min()>=-1:
        return ret_var(w)
    return (None,None,None)

In [ ]:
for runs in [1_000,5_000,10_000,50_000,100_000,500_000,1_000_000]:
    with ProcessPoolExecutor() as p:
        ports = pd.DataFrame(p.map(compute,range(runs)),columns=['ret','std','w'])
        ports['sharpe'] = (ports['ret'] - 1.6)/ports['std']
        print(runs, '------',ports.dropna().sharpe.max())

1000 ------ 5.3661922756108025
5000 ------ 5.494662763846773
10000 ------ 5.964045200168934
50000 ------ 6.1184447546726055
100000 ------ 6.170295750073656
500000 ------ 6.585020007265979
